In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
data = pd.read_csv(r'C:\Users\HUAWEI D15\Desktop\Postgrad\2.Hons SEM 2\COS711\Assignments\A2\almonds\Almond.csv')
data_numpy = data.to_numpy(None)
data_numpy = data_numpy[0:,1:]
data_numpy.shape

(2803, 13)

In [3]:
mamra = []
sanora = []
regular = []

for i in range(len(data_numpy)):
    if data_numpy[i,-1] == 'MAMRA':
        mamra.append(data_numpy[i,:])
        
    if data_numpy[i,-1] == 'SANORA':
        sanora.append(data_numpy[i,:])
        
    if data_numpy[i,-1] == 'REGULAR':
        regular.append(data_numpy[i,:])
        
mamra = np.array(mamra)
sanora = np.array(sanora)
regular = np.array(regular)

In [4]:
print(mamra.shape,sanora.shape,regular.shape)
print(len(mamra) + len(sanora) + len(regular) == len(data_numpy))

(933, 13) (943, 13) (927, 13)
True


In [5]:
All = [mamra,sanora,regular]

In [6]:
all_var = [] #variance of all features for each class
all_mean = []
all_median = []
for n in All:
    all_var.append(np.nanvar(n[:,:-1].astype(None),axis = 0))
    all_mean.append(np.nanmean(n[:,:-1].astype(None),axis = 0))
    all_median.append(np.nanmedian(n[:,:-1].astype(None),axis = 0))

In [7]:
all_mean[0][8],all_median[0][8]

(1.9230846417924634, 1.9143987085929863)

In [8]:
for n in range(len(All)):
    #Replacing NaN aspect ratios with the mean aspect ratio of that class
    All[n][:,8][np.isnan(All[n][:,8].astype(None))] = all_mean[n][8]
    
    #Replacing NaN lengths and width with one computed from aspect ratio of that class
    All[n][:,0][np.isnan(All[n][:,0].astype(None))] = All[n][:,1][np.isnan(All[n][:,0].astype(None))]*All[n][:,8][np.isnan(All[n][:,0].astype(None))]
    All[n][:,1] = All[n][:,0]/All[n][:,8]
    
    #Replacing NaN depth values with mean values for that class
    All[n][:,2][np.isnan(All[n][:,2].astype(None))] = all_mean[n][2]
    
    All[n][:,5] = 4*np.pi/(np.pi*All[n][:,0]) #Recalculating Roundness
    All[n][:,9] = np.sqrt((1 - (All[n][:,1]/All[n][:,0])**2).astype(None)) #Recalculating Eccentricity

In [9]:
for h in All:
    print(sum(np.isnan(h[:,:-1].astype(None))))

[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]


In [10]:
# with open('almonds_prep.pickle','wb') as file:
#     pickle.dump(All,file)

In [17]:
a = np.cov(All[0][:,:-1].astype(None),rowvar = False)

In [19]:
A = pd.DataFrame(a)

In [20]:
A

,0,1,2,3,4,5,6,7,8,9,10,11
0,5600.807098,2626.013354,5.131182e+02,7.134476e+05,1.217876e+04,-0.241657,-0.214417,12.232158,3.373218e+00,5.918666e-01,0.185623,7.579622e+05
1,2626.013354,1376.584874,2.668204e+02,3.497641e+05,5.692796e+03,-0.114392,-0.065119,4.929571,-4.858558e-02,1.612424e-02,0.088407,3.697113e+05
2,513.118165,266.820375,2.420249e+02,6.408291e+04,1.355510e+03,-0.022519,-0.075488,1.197638,2.600194e-28,-2.573049e-17,0.017080,7.122244e+04
3,713447.631589,349764.130860,6.408291e+04,2.913871e+08,3.843511e+06,-29.061125,169.220033,-373.472554,2.910649e+02,1.765283e+01,364.497707,3.009905e+08
4,12178.762521,5692.796436,1.355510e+03,3.843511e+06,8.085584e+04,-0.520805,-4.150915,147.734260,7.679254e+00,9.788394e-01,-0.605004,4.201160e+06
5,-0.241657,-0.114392,-2.251922e-02,-2.906113e+01,-5.208052e-01,0.000011,0.000016,-0.000626,-1.363859e-04,-2.525438e-05,-0.000003,-3.103377e+01
6,-0.214417,-0.065119,-7.548779e-02,1.692200e+02,-4.150915e+00,0.000016,0.002350,-0.037716,-6.057499e-04,-1.617831e-04,0.002329,1.091706e+02
7,12.232158,4.929571,1.197638e+00,-3.734726e+02,1.477343e+02,-0.000626,-0.037716,0.883626,1.678721e-02,3.657534e-03,-0.033339,7.822324e+02
8,3.373218,-0.048586,2.600194e-28,2.910649e+02,7.679254e+00,-0.000136,-0.000606,0.016787,2.120858e-02,3.449591e-03,0.000100,3.284533e+02
9,0.591867,0.016124,-2.573049e-17,1.765283e+01,9.788394e-01,-0.000025,-0.000162,0.003658,3.449591e-03,6.126928e-04,-0.000080,2.406517e+01
